In [1]:
import pandas as pd 
import numpy as np

In [2]:
#Open and read files for school and students csv
school_csv = "schools_complete.csv"
students_csv = "students_complete.csv"

school_df = pd.read_csv(school_csv)
students_df = pd.read_csv(students_csv)

In [3]:
#DISTRICT SUMMARY

total_schools = len(school_df["name"].unique())
total_students = students_df["name"].count()
total_budget = school_df["budget"].sum()
avg_math = students_df["math_score"].mean()
avg_reading = students_df["reading_score"].mean()

#calculate percent of students that passed based on failing score of 65%

pass_math = students_df.loc[students_df["math_score"] > 65, ["math_score"]].count()
percent_pass_math = round((pass_math/total_students) * 100, 0)

pass_reading = students_df.loc[students_df["reading_score"] > 65, ["reading_score"]].count()
percent_pass_reading = round((pass_reading/total_students) * 100, 0)

#% overall pass = students who scored >65 on math and reading
overall_pass = students_df.loc[(students_df["math_score"] > 65)  & (students_df["reading_score"] > 65), "name"].count()
percent_overall_pass = round((overall_pass/total_students) * 100, 0)

In [4]:
#creating summary table

district_summary = pd.DataFrame({"Total Schools": [total_schools],
                              "Total Students": [total_students],
                              "Total Budget": [total_budget],
                              "Average Math Score": [avg_math],
                              "Average Reading Score": [avg_reading],
                              "% Passing Math": [percent_pass_math.values[0]],
                              "% Passing Reading": [percent_pass_reading.values[0]],
                              "% Overall Passing Rate": [percent_overall_pass]})

#reorganize the format
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
organized_district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score", 
                                         "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
organized_district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,83.0,94.0,79.0


In [5]:
#SCHOOL SUMMARY

#group student csv by high schools 
school_groups = students_df.groupby(["school"])

school_df = school_df.rename(columns={"name":"High School", "type":"School Type", "size":"Total Students", "budget":"Total School Budget"})
#del school_df["School ID"]

school_df = school_df.set_index("High School")
school_df["Per Student Budget"] = (school_df["Total School Budget"]/school_df["Total Students"])
school_df["Average Math Score"] = school_groups["math_score"].mean()
school_df["Average Reading Score"] = school_groups["reading_score"].mean()

#reformat columns
school_df["Total School Budget"] = school_df["Total School Budget"].map("${:,.2f}".format)
school_df["Per Student Budget"] = school_df["Per Student Budget"].map("${:.2f}".format)

#reset index 
school_df.reset_index(inplace=True)

In [6]:
#get the necessary columns and drop all NaN
math_students_df = students_df.loc[:,["school", "math_score"]]
reading_students_df = students_df.loc[:,["school", "reading_score"]]

clean_math_students_df = math_students_df.dropna(how="all")
clean_reading_students_df = reading_students_df.dropna(how="all")

#total number of scores per high school
total_math_groups = clean_math_students_df.groupby(["school"]).count()
total_reading_groups = clean_reading_students_df.groupby(["school"]).count()

#filter to find math scores > 65 and group by school
math_scores = clean_math_students_df.loc[clean_math_students_df["math_score"] > 65]
math_groups = math_scores.groupby(["school"]).count()

#filter to find reading scores > 65 and group by school
reading_scores = clean_reading_students_df.loc[clean_reading_students_df["reading_score"] > 65]
reading_groups = reading_scores.groupby(["school"]).count()

#find percent passing math and percent passing reading
percent_math_groups = round((math_groups/total_math_groups) * 100, 0)
percent_reading_groups = round((reading_groups/total_reading_groups) * 100, 0)

#reset index to merge the two score groups
percent_math_groups.reset_index(inplace=True)
percent_reading_groups.reset_index(inplace=True)

school_scores = percent_math_groups.merge(percent_reading_groups, how = "right")
school_scores.head()

#rename columns
school_scores = school_scores.rename(columns={"school": "High School", "math_score": "% Passing Math", "reading_score": "% Passing Reading"})

#calculate overall passing rate of each high school
school_scores["% Overall Passing Rate"] = school_scores.mean(axis=1)

In [7]:
#merge "school_df" and "school_scores" tables
schools_summary = school_df.merge(school_scores, how = "right")

#set index to "High School
schools_summary = schools_summary.set_index("High School")
schools_summary

,School ID,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
High School,,,,,,,,,,
Huang High School,0,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,75.0,92.0,83.5
Figueroa High School,1,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,75.0,92.0,83.5
Shelton High School,2,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,100.0,100.0,100.0
Hernandez High School,3,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,75.0,91.0,83.0
Griffin High School,4,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,100.0,100.0,100.0
Wilson High School,5,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,100.0,100.0,100.0
Cabrera High School,6,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,100.0,100.0,100.0
Bailey High School,7,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,76.0,92.0,84.0
Holden High School,8,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,100.0,100.0,100.0


In [8]:
#TOP 5 PERFORMING SCHOOLS

top_schools = schools_summary.sort_values("% Overall Passing Rate", ascending=False)
top_schools.head(5)

,School ID,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
High School,,,,,,,,,,
Shelton High School,2,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,100.0,100.0,100.0
Griffin High School,4,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,100.0,100.0,100.0
Wilson High School,5,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,100.0,100.0,100.0
Cabrera High School,6,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,100.0,100.0,100.0
Holden High School,8,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,100.0,100.0,100.0


In [9]:
#BOTTOM 5 PERFORMING SCHOOLS

bottom_schools = schools_summary.sort_values("% Overall Passing Rate")
bottom_schools.head(5)

,School ID,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
High School,,,,,,,,,,
Hernandez High School,3,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,75.0,91.0,83.0
Huang High School,0,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,75.0,92.0,83.5
Figueroa High School,1,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,75.0,92.0,83.5
Johnson High School,12,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,75.0,92.0,83.5
Ford High School,13,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,76.0,91.0,83.5


In [10]:
#MATH SCORES BY GRADE

student_math = students_df.loc[:,["school", "grade", "math_score"]]

#group by both school and grades to find sum of math scores
grades_by_school_math = student_math.groupby(["school", "grade"])

sum_by_grade_math = grades_by_school_math.aggregate(np.sum)
sum_by_grade_math = sum_by_grade_math.reset_index()
#print(sum_by_grade_math.head())

student_count_by_grade_math = grades_by_school_math.count()
student_count_by_grade_math = student_count_by_grade_math.reset_index()
#print(student_count_by_grade_math.head())

#average math score per grade
avg_grade_math = (sum_by_grade_math["math_score"] / student_count_by_grade_math["math_score"])

#create pivot table
math_scores_grade = sum_by_grade_math[["school", "grade"]].copy()
math_scores_grade["Average Math Score"] = avg_grade_math
#math_scores_grade = math_scores_grade.rename(columns={"school": "High School", "grade": "Grade"})
math_by_grade = math_scores_grade.pivot_table("Average Math Score", ["school"], "grade")
math_by_grade = math_by_grade[["9th", "10th", "11th", "12th"]]
math_by_grade.head()

grade,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


In [11]:
#READING SCORES BY GRADE

student_reading = students_df.loc[:,["school", "grade", "reading_score"]]

#group by both school and grades to find sum of math scores
grades_by_school_reading = student_reading.groupby(["school", "grade"])

sum_by_grade_reading = grades_by_school_reading.aggregate(np.sum)
sum_by_grade_reading = sum_by_grade_reading.reset_index()
#print(sum_by_grade_reading.head())

student_count_by_grade_reading = grades_by_school_reading.count()
student_count_by_grade_reading = student_count_by_grade_reading.reset_index()
#print(student_count_by_grade_reading.head())

#average math score per grade
avg_grade_reading = (sum_by_grade_reading["reading_score"] / student_count_by_grade_reading["reading_score"])

#create pivot table
reading_scores_grade = sum_by_grade_reading[["school", "grade"]].copy()
reading_scores_grade["Average Reading Score"] = avg_grade_reading

reading_by_grade = reading_scores_grade.pivot_table("Average Reading Score", ["school"], "grade")
reading_by_grade = reading_by_grade[["9th", "10th", "11th", "12th"]]
reading_by_grade.head()

grade,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [92]:
#SCORES BY SCHOOL SPENDING

school_spending = {"Per student budget": 
                   "Average Math Score":
                   "Average Reading Score":
                   "% Passing Math":
                   "% Passing Reading":
                   "% Overall Passing Rate"}
school_spending.head()

AttributeError: 'dict' object has no attribute 'head'

In [ ]:
#SCORES BY SCHOOL SIZE

In [ ]:
#SCORES BY SCHOOL TYPE